# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Make a reference to the city data.csv file path
csv_path = "../weatherpy/city_df.csv"

# Import the city data.csv file as a DataFrame
city_df = pd.read_csv(csv_path, encoding="utf-8")
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,59.00,98,100,1.99,AU,1612655284
1,dembi dolo,8.5333,34.8000,59.56,33,46,4.03,ET,1612655659
2,tuktoyaktuk,69.4541,-133.0374,-16.60,76,90,24.16,CA,1612655283
3,hirado,33.3597,129.5531,53.01,93,40,3.44,JP,1612655666
4,codrington,-38.2667,141.9667,59.00,68,100,11.81,AU,1612655201


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
#Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]].astype(float)
weights = city_df["Humidity"].astype(float)

# Create a humidity heat layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,59.00,98,100,1.99,AU,1612655284
1,dembi dolo,8.5333,34.8000,59.56,33,46,4.03,ET,1612655659
2,tuktoyaktuk,69.4541,-133.0374,-16.60,76,90,24.16,CA,1612655283
3,hirado,33.3597,129.5531,53.01,93,40,3.44,JP,1612655666
4,codrington,-38.2667,141.9667,59.00,68,100,11.81,AU,1612655201


In [6]:
#Ideal weather conditions - max_temp less than 80 but higher than 70, Wind speed less 10 mph, and Zero cloudiness.  
new_df = city_df.loc[(city_df["Max Temp"] < 80) &\
                     (city_df["Max Temp"] > 70) &\
                     (city_df["Wind Speed"] < 10) &\
                     (city_df["Cloudiness"] == 0)]

new_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,mataura,-46.1927,168.8643,77.25,50,0,2.44,NZ,1612655229
33,tuatapere,-46.1333,167.6833,71.08,47,0,3.24,NZ,1612655269
97,bell ville,-32.6259,-62.6887,76.23,29,0,5.26,AR,1612655982
140,yenagoa,4.9247,6.2642,79.36,87,0,4.43,NG,1612656133
253,lazaro cardenas,17.9583,-102.2000,79.16,64,0,5.19,MX,1612656505


In [7]:
new_df["Hotel Name"]=""  
hotelone_df = new_df[["City", "Lat", "Lng", "Country", "Hotel Name"]]
hotelone_df.head()

<ipython-input-7-49afe221142b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Hotel Name"]=""


,City,Lat,Lng,Country,Hotel Name
17,mataura,-46.1927,168.8643,NZ,
33,tuatapere,-46.1333,167.6833,NZ,
97,bell ville,-32.6259,-62.6887,AR,
140,yenagoa,4.9247,6.2642,NG,
253,lazaro cardenas,17.9583,-102.2000,MX,


In [8]:
#Drop any rows with null values from hotel_df
hotel_df = hotelone_df.dropna(axis=0, how='all', inplace = False) 


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Create parameters to pull back hotels meeting criteria
params = {
    "radius": "5000",
    "type": "lodging",
    "keyword" : "hotels",
    "key": g_key}

# use iterrows to iterate through dataframe      
    
for index, row in hotel_df.iterrows():
    #find the closet hotels within a 5000 meter radious of the location
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #get latitude and longitude from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change the location to itterate through the cities
    params["locations"] = f"{lat},{lng}" 
    
    # assemble url and make API request
    hotel_response = requests.get(base_url, params=params)
    
    # convert to json
    hotel_response = hotel_response.json()
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = hotel_response["results"][0]['name']      
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")    

hotel_df


Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


,City,Lat,Lng,Country,Hotel Name
17,mataura,-46.1927,168.8643,NZ,
33,tuatapere,-46.1333,167.6833,NZ,
97,bell ville,-32.6259,-62.6887,AR,
140,yenagoa,4.9247,6.2642,NG,
253,lazaro cardenas,17.9583,-102.2000,MX,
267,vao,-22.6667,167.4833,NC,
296,imbituba,-28.2400,-48.6703,BR,
330,ratnagiri,16.9833,73.3000,IN,
360,santa maria,-29.6842,-53.8069,BR,
415,durazno,-33.4131,-56.5006,UY,


In [ ]:
# # params dictionary to update each iteration
# params = {
#     "radius": 50000,
#     "types": "airport",
#     "keyword": "international airport",
#     "key": gkey
# }

# # Use the lat/lng we recovered to identify airports
# for index, row in cities_pd.iterrows():
#     # get lat, lng from df
#     lat = row["Lat"]
#     lng = row["Lng"]

#     # change location each iteration while leaving original params in place
#     params["location"] = f"{lat},{lng}"

#     # Use the search term: "International Airport" and our lat/lng
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#     # make request and print url
#     name_address = requests.get(base_url, params=params)
    
# #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
# #     print(name_address.url)

#     # convert to json
#     name_address = name_address.json()
#     # print(json.dumps(name_address, indent=4, sort_keys=True))

#     # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
#     try:
#         cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, infoboxcontent = hotel_info )
fig.add_layer(markers)

# Display figure
fig
